In [1]:
from data_loader import DataLoader
from data_cleaner import DataCleaner
from feature_extractor import FeatureExtractor

from dotenv import load_dotenv
import os
load_dotenv(dotenv_path='nb-prod/.env')

True

In [2]:
data_loader = DataLoader(enstoe_api_key=os.getenv('ENTSOE_API_KEY'))
data_loader.update_df(out_df_filepath="data/bronze/df.parquet")

No data available between 2024-09-29 11:01:00+02:00 -> 2024-09-29 12:10:15.144990+02:00


In [3]:
DataCleaner.clean(
    in_df_filepath='data/bronze/df.parquet',
    out_df_filepath='data/silver/df.parquet',
)

In [ ]:
FeatureExtractor.extract_features(
    in_df_filepath='data/silver/df.parquet',
    out_df_filepath='data/gold/df.parquet',
)